<a href="https://colab.research.google.com/github/KamilBienias/data-science/blob/main/training/analityk.edu.pl/pandas_tutorial/12_unstack_i_pivot_table.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# https://analityk.edu.pl/python-pandas-tutorial-unstack-i-pivot-table/

# **Python Pandas Tutorial – unstack i pivot table**

Pivot table (czyli tabela przestawna) jest wielu osobom dobrze znane z Excela. Pandas nie pozostaje z tyłu w tym temacie. W tej lekcji pokażę czym pivot table różni różni się od grupowania, które robiliśmy za pomocą funkcji group by i jak ją zrobić. Zaczynajmy!
Zbiór który weźmiemy pod lupę jest tym samym z którego korzystaliśmy przy grupowaniu, tak więc nie będę się rozpisywać na jego temat.

In [6]:
import pandas as pd

films = pd.read_csv('https://drive.google.com/uc?export=download&id=1Ofob2EV42qPeJSpIlhxnsEMAdEvnop7l', 
    sep=';', 
    encoding = "ISO-8859-1", 
    skiprows=[1], # zaczyna od drugiego wiersza
    dtype={'Length':'float64', 'Popularity':'float64', 'Year':'int64'}, 
    usecols=['Year', 'Length', 'Title', 'Subject', 'Popularity', 'Awards'], 
    converters={'Awards':lambda x: True if x == 'Yes' else False})

films.head()

,Year,Length,Title,Subject,Popularity,Awards
0,1990,111.0,Tie Me Up! Tie Me Down!,Comedy,68.0,False
1,1991,113.0,High Heels,Comedy,68.0,False
2,1983,104.0,"Dead Zone, The",Horror,79.0,False
3,1979,122.0,Cuba,Action,6.0,False
4,1978,94.0,Days of Heaven,Drama,14.0,False


## **Grupowanie dwóch kolumn, z group by**

Zanim przejdziemy do pivot tables, spójrzmy na jeden przypadek. Mianowicie powiedzmy że chcemy pogrupować dane po dwóch kolumnach. Niech to będzie Year oraz Subject. Dla każdej grupy chcemy policzyć średnią popularność:

In [8]:
films.groupby(["Year", "Subject"]).agg(avg_popularity=("Popularity", "mean"))

avg_popularity
Year Subject                        
1920 Drama                     29.00
1923 Western                     NaN
1924 Drama                     72.00
1925 Drama                     73.00
1926 Action                    55.00
...                              ...
1993 Music                     48.00
     Mystery                   38.00
     Western                   61.25
1996 Horror                    39.00
1997 Science Fiction           60.00

[361 rows x 1 columns]

## **Czym jest pivot table?**

Pivot table, czyli tak zwana tabela przestawna, jest odmianą tabeli, którą otrzymujemy w wyniku grupowania, różniącą się innym układem kolumn.

Różnica polega na tym, że wybrana przez nas kolumna, zamienia się w szereg kolumn. W naszym przypadku, możemy zamienić naszą tabelę tak, aby kolumna zawierająca kategorię filmu stworzyła szereg kolumn, gdzie każda będzie odpowiadać konkretnej kategorii. Zobaczmy:

## **Pivot table z funkcją unstack()**

Najprostszym sposobem utworzenia pivot table, jest zastosowanie funkcji unstack() na naszym wynikowym zbiorze grupowania:

In [11]:
x = films.groupby(['Year','Subject']).agg(avg_popularity = ('Popularity','mean'))
x.unstack()

avg_popularity                       ...                           
Subject         Action Adventure     Comedy  ...    War    Western Westerns
Year                                         ...                           
1920               NaN       NaN        NaN  ...    NaN        NaN      NaN
1923               NaN       NaN        NaN  ...    NaN        NaN      NaN
1924               NaN       NaN        NaN  ...    NaN        NaN      NaN
1925               NaN       NaN        NaN  ...    NaN        NaN      NaN
1926         55.000000       NaN        NaN  ...    NaN        NaN      NaN
...                ...       ...        ...  ...    ...        ...      ...
1991         38.937500       NaN  42.540541  ...  50.75  44.888889     22.5
1992         36.944444      39.0  51.333333  ...   3.00  28.500000     68.0
1993         32.000000       NaN  38.428571  ...    NaN  61.250000      NaN
1996               NaN       NaN        NaN  ...    NaN        NaN      NaN
1997               NaN       NaN        NaN  ...    NaN        NaN      NaN

[74 rows x 15 columns]

Druga kolumna w naszym zbiorze wynikowym z group by, utworzyła kilka kolumn. Są to dokładnie te same wynik co wcześniej, jednak inaczej przedstawione. O wiele, wiele czytelniej. Możemy wzrokiem w łatwy sposób przeglądać dane po roku, jak i po kategorii. Co było bardzo trudne w przypadku kiedy mieliśmy sam wynik grupowania.

Magii brak, natomiast czytelność się drastycznie zwiększyła. Stąd też popularność tabel przestawnych.

## **Pandas Pivot table, z funkcją…. pivot_table()**

Biblioteka Pandas dostarcza również dedykowaną funkcję do tego celu, która przyjmuje kilka nowych parametrów, takich jak:

index – czyli która kolumna ma być po lewej stronie w wierszach, jako nasz index

columns – która kolumna ma utworzyć szereg nowych kolumn, na górze

values – wynik operacji na której kolumnie ma być widoczny 'wewnątrz' naszej tabeli przestawnej

aggfunc – jaką operację chcemy przeprowadzić – mean, sum, count, max, min itp

Najprostsza postać wywołania tej funkcji, która daje nam dokładnie takie same wyniki jak funkcje group_by() oraz unstack(), może wyglądać tak :

In [12]:
pd.pivot_table(films, 
               index='Year', 
               columns='Subject', 
               values='Popularity',
               aggfunc='mean')

Subject,Action,Adventure,Comedy,Crime,Drama,Fantasy,Horror,Music,Mystery,Romance,Science Fiction,Short,War,Western,Westerns
Year,,,,,,,,,,,,,,,
1920,NaN,NaN,NaN,NaN,29.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1924,NaN,NaN,NaN,NaN,72.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1925,NaN,NaN,NaN,NaN,73.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926,55.000000,NaN,NaN,NaN,72.000000,NaN,NaN,NaN,76.0,NaN,49.0,NaN,NaN,NaN,NaN
1927,NaN,NaN,NaN,NaN,31.000000,NaN,NaN,NaN,45.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,38.937500,NaN,42.540541,NaN,41.900000,NaN,41.6,53.714286,37.0,NaN,11.0,NaN,50.75,44.888889,22.5
1992,36.944444,39.0,51.333333,NaN,48.566667,NaN,NaN,24.400000,39.2,NaN,36.5,NaN,3.00,28.500000,68.0
1993,32.000000,NaN,38.428571,NaN,33.750000,NaN,NaN,48.000000,38.0,NaN,NaN,NaN,NaN,61.250000,NaN


## **Pandas GroupBy + unstack vs pivot_table**

To skoro mamy co najmniej 2 metody tworzenia tabel przestawnych, to którą mam stosować?

Tą którą Ci wygodniej. Osobiście, w większości przypadków stosuję groupby + unstack. Ponieważ łatwiej mi wykonać operacje dodatkowe na tabeli będącej wynikiem grouby, a na samym końcu zamieniań na pivot table.

Przykładowo. Trzymajmy się policzenia średniej popularności dla filmów per kategoria. Jeżeli popularność jest większa niż 60, to chciałbym zamiast wartości wstawić 'Bardzo popularny', natomiast jak jest mniej punktów to wstawić 'Niszowy'. Moje rozwiązanie przy użyciu groupby byłoby takie:

In [13]:
result = films.groupby(['Year','Subject']).agg(avg_popularity = ('Popularity','mean'))
result = result.avg_popularity.apply(lambda x: 'Bardzo popularny' if x > 60 else 'Niszowy')
result.unstack(fill_value='')

Subject,Action,Adventure,Comedy,Crime,Drama,Fantasy,Horror,Music,Mystery,Romance,Science Fiction,Short,War,Western,Westerns
Year,,,,,,,,,,,,,,,
1920,,,,,Niszowy,,,,,,,,,,
1923,,,,,,,,,,,,,,Niszowy,
1924,,,,,Bardzo popularny,,,,,,,,,,
1925,,,,,Bardzo popularny,,,,,,,,,,
1926,Niszowy,,,,Bardzo popularny,,,,Bardzo popularny,,Niszowy,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1991,Niszowy,,Niszowy,,Niszowy,,Niszowy,Niszowy,Niszowy,,Niszowy,,Niszowy,Niszowy,Niszowy
1992,Niszowy,Niszowy,Niszowy,,Niszowy,,,Niszowy,Niszowy,,Niszowy,,Niszowy,Niszowy,Bardzo popularny
1993,Niszowy,,Niszowy,,Niszowy,,,Niszowy,Niszowy,,,,,Bardzo popularny,


Dla mnie, zrobienie pivota jest ostanim elementem. Ale to już być może kwestia gustu.

## **Podsumowując nasze pivot_table czyli table przestawne w Python Pandas**

Tabela przestawna może być wynikiem prostego zabiegu na wyniku grupowania, lub też efektem użycia funkcji pivot_table. Jak komu wygodniej. Wyniki są takie same. Na koniec końców liczy się poprawność danych oraz atrakcyjny przekaz, który jest mocno związany z czytelnością.

## **Ćwiczenia z dataframe pivot table**

1. Wyświetlić tabelę z Kategoriami filmu w wierszach, informację o zdobytej nagrodzie w kolumnach, natomiast w komórkach z ilością zdobytych nagród

2. Policzyć ilość filmów, dla roku oraz kategorii filmu

In [17]:
# 1.
pd.pivot_table(films, 
               index='Subject', 
               columns='Year', 
               values='Awards',
               aggfunc='count')

Year,1920,1923,1924,1925,1926,1927,1928,1929,1930,1931,1932,1933,1934,1935,1936,1937,1938,1939,1940,1941,1942,1943,1944,1945,1946,1947,1948,1949,1950,1951,1952,1953,1954,1955,1956,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1996,1997
Subject,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Action,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,NaN,3.0,1.0,NaN,1.0,2.0,1.0,1.0,2.0,1.0,2.0,NaN,2.0,NaN,1.0,2.0,2.0,2.0,3.0,2.0,2.0,4.0,4.0,6.0,3.0,3.0,3.0,8.0,2.0,3.0,5.0,4.0,8.0,4.0,6.0,16.0,14.0,19.0,17.0,16.0,18.0,2.0,NaN,NaN
Adventure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN
Comedy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,3.0,2.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,2.0,NaN,2.0,2.0,1.0,6.0,1.0,2.0,4.0,2.0,NaN,1.0,1.0,4.0,3.0,4.0,6.0,5.0,4.0,2.0,8.0,2.0,7.0,4.0,2.0,6.0,3.0,5.0,4.0,12.0,10.0,6.0,8.0,17.0,11.0,15.0,41.0,28.0,26.0,28.0,26.0,37.0,9.0,7.0,NaN,NaN
Crime,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Drama,1.0,NaN,3.0,1.0,1.0,2.0,4.0,4.0,2.0,5.0,3.0,3.0,2.0,2.0,1.0,2.0,3.0,5.0,5.0,3.0,4.0,4.0,2.0,4.0,5.0,2.0,6.0,5.0,4.0,3.0,6.0,7.0,6.0,9.0,9.0,10.0,5.0,3.0,6.0,10.0,7.0,7.0,5.0,5.0,8.0,5.0,7.0,8.0,10.0,13.0,11.0,10.0,13.0,7.0,8.0,13.0,6.0,6.0,10.0,12.0,20.0,13.0,17.0,17.0,35.0,42.0,40.0,31.0,33.0,40.0,30.0,4.0,NaN,NaN
Fantasy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Horror,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,2.0,4.0,1.0,1.0,2.0,2.0,2.0,3.0,NaN,NaN,4.0,2.0,5.0,7.0,5.0,5.0,NaN,NaN,1.0,NaN
Music,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,1.0,1.0,NaN,NaN,1.0,1.0,1.0,2.0,NaN,1.0,1.0,1.0,NaN,1.0,NaN,2.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,5.0,7.0,5.0,1.0,NaN,NaN
Mystery,NaN,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,1.0,2.0,2.0,1.0,NaN,1.0,NaN,NaN,1.0,NaN,NaN,3.0,NaN,2.0,NaN,1.0,1.0,NaN,NaN,2.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,3.0,NaN,NaN,1.0,NaN,2.0,NaN,1.0,1.0,NaN,NaN,1.0,2.0,1.0,2.0,1.0,NaN,1.0,NaN,2.0,4.0,6.0,7.0,10.0,8.0,8.0,15.0,3.0,NaN,NaN


In [21]:
# 2.
result = films.groupby(["Year", "Subject"]).agg(ilosc_filmow=("Subject", "count"))
result

ilosc_filmow
Year Subject                      
1920 Drama                       1
1923 Western                     1
1924 Drama                       3
1925 Drama                       1
1926 Action                      1
...                            ...
1993 Music                       1
     Mystery                     3
     Western                     4
1996 Horror                      1
1997 Science Fiction             1

[361 rows x 1 columns]